In [2]:
import tiktoken
from datasets import load_dataset

/opt/homebrew/Caskroom/miniconda/base/envs/learning-pytroch-compoutervision/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
ds = load_dataset("HuggingFaceFW/fineweb", split="train", streaming=False)
shuffled_dataset = ds.shuffle(buffer_size=100000)

KeyboardInterrupt: 

In [3]:
enc = tiktoken.encoding_for_model("gpt-4o")

In [4]:
for i in shuffled_dataset:
    print(i)
    print(enc.encode(str(i["text"])))
    break

KeyboardInterrupt: 

In [4]:
import numpy as np 

np.empty((128))

2.134875756e-314

In [12]:
import os 



In [31]:
random_integers = np.random.random_integers(0, len(shards), 10)
data = []
for i in random_integers:
    data.extend(np.load(f"./fineweb10B/{shards[i]}"))

/var/folders/tf/g3md_wds3j183knw90n18sdm0000gn/T/ipykernel_83728/577354194.py:1: DeprecationWarning: This function is deprecated. Please call randint(0, 99 + 1) instead
  random_integers = np.random.random_integers(0, len(shards), 10)


In [32]:
len(data)

1000000000

In [62]:
import torch

def load_tokens(filename):
    npt = np.load(filename)
    npt = npt.astype(np.int32)
    ptt = torch.tensor(npt, dtype=torch.long)
    return ptt

class DataLoaderLite:
    def __init__(self, B, T, split):
        self.B = B
        self.T = T
        assert split in {'train', 'val'}

        data_root = "fineweb10B"
        shards = os.listdir(data_root)
        shards = [s for s in shards if split in s]
        shards = sorted(shards)
        shards = [os.path.join(data_root, s) for s in shards]
        self.shards = shards
        assert len(shards) > 0, f"no shards found for split {split}"
        print(f"found {len(shards)} shards for split {split}")
        self.reset()

    def reset(self):
        # state, init at shard zero
        self.current_shard = 0
        self.tokens = load_tokens(self.shards[self.current_shard])
        self.current_position = self.B * self.T

    def next_batch(self):
        B, T = self.B, self.T
        buf = self.tokens[self.current_position : self.current_position+B*T+1]
        x = (buf[:-1]).view(B, T) # inputs
        y = (buf[1:]).view(B, T) # targets
        # advance the position in the tensor
        self.current_position += B * T
        # if loading the next batch would be out of bounds, advance to next shard
        if self.current_position + (B * T + 1) > len(self.tokens):
            self.current_shard = (self.current_shard + 1) % len(self.shards)
            self.tokens = load_tokens(self.shards[self.current_shard])
            self.current_position = B * T
        return x, y

In [63]:
B = 16
T = 2048
train_loader = DataLoaderLite(B=B, T=T, split="train")

found 103 shards for split train


In [66]:
x, y = train_loader.next_batch()

In [71]:
max(x[3])

tensor(50256)

In [72]:
y.shape

torch.Size([16, 2048])